In [1]:
import os; os.chdir("/home/ubuntu/coma/")
import tensorflow as tf
import numpy as np
import ipywidgets as widgets
import yaml
from copy import deepcopy
import json
import re
import itertools
import scipy
import helpers

from scipy.spatial import procrustes as procrustes

import glob, os, json
import pandas as pd

from main import generate_model #, run as train
from VTK.VTKMesh import VTKObject as Mesh
from cardiac_mesh import CardiacMesh

## Definitions of paths and constants

In [2]:
meshes_dir = "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/"
output_dir = "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/"
checkpoints_dir = "{}/checkpoints".format(output_dir)

subparts_ids = CardiacMesh.subparts_ids

## Collect runs

In [5]:
runs = helpers.RunCollection(output_dir)
# kk = runs.run_info['2020-04-15_00_29_43_203151']
params_df = runs.get_params_df()
params_df.to_csv("/MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/run_parameters.csv", index_label="run_id")

,ds_factors,hash,regularization,data_dir,decay_rate,nz,K,F,learning_rate,which_loss,dir_name,partition,nTraining
2020-04-14_20_37_05_811720,"[4, 4, 2, 2]",701744,0.0001,/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/n...,0.990,8,"[6, 6, 6, 6]","[16, 16, 16, 32]",0.010,l1,LV__ds_4422__nz_8__reg_0.0001__lr_0.01,LV,1600
2020-04-15_00_29_43_203151,"[4, 4, 2, 2]",924100,0.0001,None,0.990,8,"[6, 6, 6, 6]","[16, 16, 16, 32]",0.010,l1,LV__ds_4422__nz_8__reg_0.0001__lr_0.01,LV,1600
2020-04-15_01_18_47_829636,"[4, 4, 2, 2]",489252,0.0001,/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/n...,0.990,8,"[6, 6, 6, 6]","[16, 16, 16, 32]",0.001,l1,LV__ds_4422__nz_8__reg_0.0001__lr_0.001,LV,1600
2020-04-19_03_11_19_339030,"[4, 4, 2, 2]",471716,0.0001,None,0.990,16,"[6, 6, 6, 6]","[16, 16, 16, 32]",0.010,l1,RV__ds_4422__nz_16__reg_0.0001__lr_0.01,RV,1600
2020-04-19_04_33_21_601306,"[4, 4, 2, 2]",77176,0.0001,None,0.990,4,"[6, 6, 6, 6]","[16, 16, 16, 32]",0.001,l1,RV__ds_4422__nz_4__reg_0.0001__lr_0.001,RV,1600
2020-04-19_05_55_19_892678,"[4, 4, 2, 2]",365302,0.0001,None,0.990,8,"[6, 6, 6, 6]","[16, 16, 16, 32]",0.001,l1,RV__ds_4422__nz_8__reg_0.0001__lr_0.001,RV,1600
2020-04-19_07_17_23_194531,"[4, 4, 2, 2]",511530,0.0001,None,0.990,8,"[6, 6, 6, 6]","[16, 16, 16, 32]",0.010,l1,RV__ds_4422__nz_8__reg_0.0001__lr_0.01,RV,1600
2020-04-19_08_39_27_313505,"[4, 4, 2, 2]",668832,0.0001,None,0.990,16,"[6, 6, 6, 6]","[16, 16, 16, 32]",0.001,l1,RV__ds_4422__nz_16__reg_0.0001__lr_0.001,RV,1600
2020-04-19_10_01_33_510833,"[4, 4, 2, 2]",17564,0.0001,None,0.990,4,"[6, 6, 6, 6]","[16, 16, 16, 32]",0.010,l1,RV__ds_4422__nz_4__reg_0.0001__lr_0.01,RV,1600
2020-04-24_09_54_28_706915,"[4, 4, 2, 2]",540220,0.0001,None,0.990,16,"[6, 6, 6, 6]","[16, 16, 16, 32]",0.001,l1,LV__ds_4422__nz_16__reg_0.0001__lr_0.001,LV,1600


## Evaluate performance of models and obtain reconstructed meshes
This cell iterates through the different models trained and 
1) loads the training and testing datasets,
2) creates the latent codes for each of the meshes and saves them into output/latent_codes/<FILENAME>.csv, 
3) reconstructs the meshes and saves them into output/test_set_assessment/<FILENAME>.npy and 
4) computes the reconstruction error and produces a file summarizing the performance for each model.

In [5]:
# This line is called each time because the train_file changes
# MM = CardiacMesh(
#   nVal = config["nVal"],
#   nTraining = config["nTraining"],
#   train_file = os.path.join(config["data_dir"], "train.npy"),
#   test_file = os.path.join(config["data_dir"], "test.npy"),
#   reference_mesh_file = config["reference_mesh_file"],
#   subpart = subparts_ids[config["partition"]]
# )

# for model_id in runs_info:
#     
#     run_info = runs_info[model_id]
#     params = deepcopy(run_info['params'])
#     
#     nTraining = params["nTraining"]
#     nVal = params["nVal"]
#     
#     train_ids_ = [ x.strip() for x in open(os.path.join(run_info['params']['data_dir'], "train_ids.txt")) ]
#     
#     train_ids = train_ids_[:nTraining]
#     val_ids = train_ids_[nTraining:(nTraining+nVal)]
#     test_ids  = [ x.strip() for x in open(os.path.join(run_info['params']['data_dir'], "test_ids.txt")) ]
#     
#     kk = _assign_set(all_ids, run_info)
#     
#     train_ids = [ x for x in kk if kk[x] == "training" ]
#     val_ids = [ x for x in kk if kk[x] == "validation" ]
#     test_ids  = [ x for x in kk if kk[x] == "testing" ]
#         
#     train_ids_file = os.path.join(params['checkpoint_dir'], "{}_training_ids.txt".format(model_id))
#     val_ids_file = os.path.join(params['checkpoint_dir'], "{}_validation_ids.txt".format(model_id))
#     test_ids_file = os.path.join(params['checkpoint_dir'], "{}_testing_ids.txt".format(model_id))
#     
#     open(train_ids_file, "w").write("\n".join(train_ids))
#     open(val_ids_file, "w").write("\n".join(val_ids))
#     open(test_ids_file, "w").write("\n".join(test_ids))

In [6]:
perf_dfs = []

previous_mesh_data = None

for model_id, run_info in runs.run_info.items():
                    
    params = deepcopy(run_info.params) # I need to deepcopy because I delete the 'seed' key
    del params['seed']
    
    try:
        model = generate_model(params, add_hash=False)
        model.dir_name = os.path.basename(run_info.checkpoints) # dir_name + "__" + model_id    
    
        if params['data_file'] != previous_mesh_data:
            print("Loading {}".format(params['data_file']))
            all_meshes = np.load(params['data_file'], allow_pickle=True)
            all_ids = [x.strip() for x in open(params['ids_file'])]
            # Preprocess meshes (procrustes-aligning + centering + scaling)
            meshes_aligned = np.array([ 
                procrustes(
                    all_meshes[i], 
                    model.reference_mesh.points)[0] for i in range(len(all_meshes)
                )
            ])
            
            meshes_aligned = (meshes_aligned - model.mean) / model.std
            previous_mesh_data = params['data_file']
    except:
        continue
    
    # Compute latent space and save it into a csv file
    z = model.encode(data=meshes_aligned)
    z_columns = ["z" + str(i) for i in range(z.shape[1])]
    pd.DataFrame(
        z, all_ids, z_columns
    ).to_csv(run_info.latent_space, index_label="IID")    
       
        
    # Reconstruct mesh from latent space and save it into a npy file
    meshes_reconstructed = model.decode(z)
    
    mse = ((meshes_aligned - meshes_reconstructed)**2).mean(axis=1).mean(axis=1)
    
    shuffled_ids = range(meshes_reconstructed.shape[0])
    np.random.shuffle(shuffled_ids)        
    mse_shuffled = ((meshes_aligned - meshes_reconstructed[shuffled_ids,:,:])**2).mean(axis=1).mean(axis=1)
    
    # from IPython import embed; embed()
    
    perf_df = pd.DataFrame({
        "model_id": [model_id] * len(all_ids),
        "subject_id": all_ids, 
        "dataset": [run_info.subset[x] for x in all_ids], 
        "mse": mse,
        "mse_shuffled": mse_shuffled,
        "n": [z.shape[1]] * len(all_ids)
    })     
    
    perf_df.to_csv(run_info.performance_summary, index=False)
    
    perf_dfs.append(perf_df)    
           
    meshes_reconstructed = meshes_reconstructed * model.std + model.mean                            

    np.save(run_info.reconstructed_meshes, meshes_reconstructed)

Loading data .. 
Vertices normalized
Generating Transform Matrices ..


lib/mesh_sampling.py:71: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  tmp_coeffs = np.linalg.lstsq(A, target.v[i])[0]
lib/mesh_sampling.py:67: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coeffs_v[3 * i:3 * i + 3] = np.linalg.lstsq(A, nearest_v)[0]


Computing Graph Laplacians ..
NN architecture
Loading /MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/LV_all_subjects/train.npy
INFO:tensorflow:Restoring parameters from /MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/checkpoints/LV__ds_4422__nz_8__reg_0.0001__lr_0.05__2020-04-26_04_14_16_329532/model-50000
INFO:tensorflow:Restoring parameters from /MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/checkpoints/LV__ds_4422__nz_8__reg_0.0001__lr_0.05__2020-04-26_04_14_16_329532/model-50000
Loading data .. 
Vertices normalized
Generating Transform Matrices ..
Computing Graph Laplacians ..
NN architecture
INFO:tensorflow:Restoring parameters from /MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/checkpoints/LV__ds_4422__nz_8__reg_0.0001__lr_0.001__2020-04-26_16_38_41_787226/model-50000
INFO:tensorflow:Restoring parameters from /MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/checkpoints/LV__ds_4422__nz_8__reg_0.0001__lr_0.001__2020-04-26_16_38_41_787226/model-50000
Loading data .. 
Vertices normali

NN architecture
Loading /MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/RV/RV_all.npy
INFO:tensorflow:Restoring parameters from /MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/checkpoints/RV__ds_22222__nz_8__reg_0.0005__lr_0.001__2020-04-30_17_29_17_136557/model-50000
INFO:tensorflow:Restoring parameters from /MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/checkpoints/RV__ds_22222__nz_8__reg_0.0005__lr_0.001__2020-04-30_17_29_17_136557/model-50000
Loading data .. 
Vertices normalized
Generating Transform Matrices ..
Computing Graph Laplacians ..
NN architecture
Loading /MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/LV_all_subjects/train.npy
INFO:tensorflow:Restoring parameters from /MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/checkpoints/LV__ds_4422__nz_8__reg_0.0001__lr_0.01__2020-04-26_13_15_04_068652/model-50000
INFO:tensorflow:Restoring parameters from /MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/checkpoints/LV__ds_4422__nz_8__reg_0.0001__lr_0.01__2020-04-26_13_15_04_0

INFO:tensorflow:Restoring parameters from /MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/checkpoints/RV__ds_4422__nz_16__reg_0.0001__lr_0.01__2020-04-19_03_11_19_339030/model-50000
INFO:tensorflow:Restoring parameters from /MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/checkpoints/RV__ds_4422__nz_16__reg_0.0001__lr_0.01__2020-04-19_03_11_19_339030/model-50000
Loading data .. 
Vertices normalized
Generating Transform Matrices ..
Computing Graph Laplacians ..
NN architecture
INFO:tensorflow:Restoring parameters from /MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/checkpoints/RV__ds_2244__nz_8__reg_0.0001__lr_0.01__2020-04-30_14_35_56_319259/model-46200
INFO:tensorflow:Restoring parameters from /MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/checkpoints/RV__ds_2244__nz_8__reg_0.0001__lr_0.01__2020-04-30_14_35_56_319259/model-46200
Loading data .. 
Vertices normalized
Generating Transform Matrices ..
Computing Graph Laplacians ..
NN architecture
Loading /MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/n

INFO:tensorflow:Restoring parameters from /MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/checkpoints/RV__ds_2244__nz_8__reg_0.0001__lr_0.01__2020-04-30_14_35_57_366833/model-50000
INFO:tensorflow:Restoring parameters from /MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/checkpoints/RV__ds_2244__nz_8__reg_0.0001__lr_0.01__2020-04-30_14_35_57_366833/model-50000
Loading data .. 
Vertices normalized
Generating Transform Matrices ..
Computing Graph Laplacians ..
NN architecture
INFO:tensorflow:Restoring parameters from /MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/checkpoints/RV__ds_4422__nz_8__reg_0.0001__lr_0.001__2020-04-19_05_55_19_892678/model-50000
INFO:tensorflow:Restoring parameters from /MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/checkpoints/RV__ds_4422__nz_8__reg_0.0001__lr_0.001__2020-04-19_05_55_19_892678/model-50000
Loading data .. 
Vertices normalized
Generating Transform Matrices ..
Computing Graph Laplacians ..
NN architecture
INFO:tensorflow:Restoring parameters from /MULTIX/DATA

ValueError: Can't load save_path when it is None.

In [7]:
perf_df_all = pd.concat(perf_dfs)
perf_df_all.to_csv("/MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/performance_summary/runs_performance_summary.csv", index=False)